# Iteración de política:(del libro de Sutton)

<img src="iteracion de politica.PNG">


In [6]:
import numpy as np
import pprint
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [7]:
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()

In [8]:
# Del ejercicio de evaluación de política

def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluar una política dado un ambiente y una descripción completa
    de la dinámica del ambiente.
    
    Argumentos:
        policy: matriz de tamaño [S, A] representando la política.
        env: ambiente de OpenAI representadno las probabilidades de transición
        del ambiente. 
        env.P[s][a] es una lista de vectores (probabilidad, próximo_estado, recompensa, done)
        env.nS es el número de estados en el ambiente
        env.nA es el número de acciones en el ambiente
        theta: para la evaluación de la política una vez que la función de valor cambia menos que
        theta para todos los estados
        discount_factor: factor de descuento gama.
        
    Retorna:
        Vector de longitud env.nS que representa la función de valor.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    while True:
        delta = 0
        # For each state, perform a "full backup"
        for s in range(env.nS):
            v = 0
            # Look at the possible next actions
            for a, action_prob in enumerate(policy[s]):
                # For each action, look at the possible next states...
                for  prob, next_state, reward, done in env.P[s][a]:
                    # Calculate the expected value
                    v += action_prob * prob * (reward + discount_factor * V[next_state])
            # How much our value function changed (across any states)
            delta = max(delta, np.abs(v - V[s]))
            V[s] = v
        # Stop evaluating once our value function change is below a threshold
        if delta < theta:
            break
    return np.array(V)

In [9]:
def policy_improvement(env, policy_eval_fn=policy_eval, discount_factor=1.0):
    """
    Algoritmo de mejora de una política. Evalúa iterativamente y mejora una política 
    hasta que encuentra la política óptima.
    
    Args:
        env: ambiente de OpenAI.
        policy_eval_fn: función de evaluación de política que toma 3 argumentos: policy, env, discount_factor
        discount_factor: factor de descuento gama
        
    Retorna:
        Un tuple (policy, V)
        A tuple (policy, V). 
        policy es la política óptima, una matriz de tamaño [S, A] en que cada estado s contiene una distribución de probabilidad 
        valida sobre el espacio de acciones.
        V es la función de valor para la política óptima.
        
    """

    def one_step_lookahead(state, V):
        """
        Función auxiliar para calcular el valor de todas las acciones dado un estado
        Helper function to calculate the value for all action in a given state.
        
        Argumentos:
            state: es estado a considerar
            V: La función de valor a utilizar, un vector de longitud env.nS
        
        Retorna:
            Un vector de longitud env.nA conteniendo el valor esperado de cada acción.
        """
        A = np.zeros(env.nA)
        for a in range(env.nA):
            for prob, next_state, reward, done in env.P[state][a]:
                A[a] += prob * (reward + discount_factor * V[next_state])
        return A
    
    # Comenzar con política aleatoria
    policy = np.ones([env.nS, env.nA]) / env.nA
    
    while True:
        # evaluar (i.e. calcular V, la función de valor de) la política actual
        V = policy_eval_fn(policy, env, discount_factor)
        
        # mantener una variable que indique si la política es estable (no cambió en este paso)
        policy_stable = True
        
        # por cada estado:
        for s in range(env.nS):
            # encontrar cuál es la acción que tomaría la política actual con más alta probabilidad
            chosen_a = np.argmax(policy[s])
            
            # calcular el valor esperado de cada acción utilizando la función de valor actual y haciendo 'one-step look-ahead'
            # encontrar la acción con mayor valor esperado dado del cálculo anterior
            action_values = one_step_lookahead(s, V)
            best_a = np.argmax(action_values)
            
             # si la acción de la política actual no coincide con la mejor calculada
            if chosen_a != best_a:
                # marcar que la política no fue estable en este paso
                policy_stable = False
            # actualizar la política 
            policy[s] = np.eye(env.nA)[best_a]
        
        # si la política es estable, devolver la política óptima y la función de valor de esa política
        if policy_stable:
            return policy, V

In [10]:
policy, v = policy_improvement(env)
print("Distribución de probabilidad de la política:")
print(policy)
print("")

print("Política exhibida la grilla: (0=arriba, 1=derecha, 2=abajo, 3=izquierda)")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Función de valor:")
print(v)
print("")

print("Función de valor exhibida en la grilla:")
print(v.reshape(env.shape))
print("")


Distribución de probabilidad de la política:
[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]

Política exhibida la grilla: (0=arriba, 1=derecha, 2=abajo, 3=izquierda)
[[0 3 3 2]
 [0 0 0 2]
 [0 0 1 2]
 [0 1 1 0]]

Función de valor:
[ 0. -1. -2. -3. -1. -2. -3. -2. -2. -3. -2. -1. -3. -2. -1.  0.]

Función de valor exhibida en la grilla:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]



In [11]:
# Test de la función de valor
expected_v = np.array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)